# Accessing here technologies
https://developer.here.com/

## Get Lat/Lon

In [711]:
import requests
import bs4
import html

def GetLatLon(location):
    app_id = ''
    app_code = ''
    location = location.replace(" ", "+")
    URL = "https://geocoder.cit.api.here.com/6.2/geocode.xml?app_id="+app_id+"&app_code="+app_code+"&searchtext="+location
    r = requests.get(url=URL)
    try:
        soup = bs4.BeautifulSoup(r.text, 'lxml')
        soup = soup.body.response.navigationposition.contents
        lat = soup[0].contents[0]
        lon = soup[1].contents[0]
        return lat, lon
    except:
        return None

In [712]:
GetLatLon('425 W Randolph St, Chicago, IL 60606, United States')

('41.88449', '-87.6387699')

## Places API Calls
    1) hospital-health-care-facility
    2) Snacks-fast-food
    3) toilet-rest-area
    4) Food Banks (https://data.seattle.gov/Community/Food-Banks/ryz5-i54h)
    5) Meals Programs in Seattle (https://data.seattle.gov/Community/Meals-Programs-in-Seattle/hmzu-x5ed/data)
    6) Libraries (https://data.seattle.gov/Community/Libraries/usk4-ujha)
    7) Family Support Centers (https://data.seattle.gov/Community/Family-Support-Center/cy6k-d7r5)
    8) Shelters
    9) Free Wifi hotspots (Excel table) + data base for libraries
    10) Churches
    11) Education (Excel table) including english classes, computer literacy,etc

In [935]:
import json 

def GetPlace(lat, lon, place):
    place_dict = {}
    place = place.replace(" ", "+")
    app_id = ''
    app_code = ''
    URL = "https://places.demo.api.here.com/places/v1/discover/search?at="+lat+"%2C"+lon+"&q="+place+"&tf=html&app_id="+app_id+"&app_code="+app_code
    r = requests.get(url=URL)
    soup = json.loads(bs4.BeautifulSoup(r.text, 'lxml').html.body.get_text())
    for i in range(len(soup['results']['items'])):
        destination = soup['results']['items'][i]['position'] #vicinity
        distance, time = GetRoute(lat, lon, str(destination[0]), str(destination[1]))
        place_dict[soup['results']['items'][i]['title']] = (distance, time, destination)
    if len(place_dict) > 5:
        min_time = sorted(place_dict, key=place_dict.get, reverse=False)[:5]
        place_dict = {x: place_dict[x] for x in min_time}
    else:
        place_dict = place_dict
    print(place_dict)

In [936]:
lat, lon = GetLatLon('425 W Randolph St, Chicago, IL 60606, United States')
GetPlace(lat, lon, 'mental health')

{'Mental Health America Of Il': (0.8, 21.3, [41.88606, -87.625535]), 'Mental Health Association Of Greater Chicago': (0.7, 19.4, [41.879944, -87.630645]), 'Illinois Institute for Integrity Mental Health': (1.0, 26.1, [41.889432, -87.624945])}


## Routing

In [893]:
def GetRoute(lat,lon,lat1,lon1):
    app_id = ''
    app_code = ''
    mode='fastest%3Bpedestrian'
    URL = "https://route.cit.api.here.com/routing/7.2/calculateroute.xml?app_id="+app_id+"&app_code="+app_code+"&waypoint0="+lat+"%2C"+lon+"&waypoint1="+lat1+"%2C"+lon1+"&mode="+mode
    r = requests.get(url=URL)
    soup = bs4.BeautifulSoup(r.text, 'lxml')
    try:
        time = round(int(soup.body.response.traveltime.contents[0])/60, 1)
        distance = round(int(soup.body.response.distance.contents[0])/1609, 1)
        return distance, time
    except:
        return 99,99

In [894]:
def GetMapRoute(lat,lon,lat1,lon1):
    app_id = ''
    app_code = ''
    lc='1652B4'
    lw='6'
    t='0'
    ppi='320'
    w='400'
    h='600'
    URL = "https://image.maps.cit.api.here.com/mia/1.6/routing?app_id="+app_id+"&app_code="+app_code+"&waypoint0="+lat+"%2C"+lon+"&waypoint1="+lat1+"%2C"+lon1+"&lc="+lc+"&lw="+lw+"&t="+t+"&ppi="+ppi+"&w="+w+"&h="+h
    r = requests.get(url=URL)
    soup = bs4.BeautifulSoup(r.text, 'xml')
    print(URL)

In [895]:
lat1, lon1 = GetLatLon('4219 42nd Ave NE, Seattle, WA 98105')
lat2, lon2 = GetLatLon('4240 Roosevelt Seattle')

GetRoute(lat1, lon1, lat2, lon2)

(1.8, 48.3)

In [754]:
GetMapRoute(lat1, lon1, lat2, lon2)

https://image.maps.cit.api.here.com/mia/1.6/routing?app_id=IYPjp4rMqRSliStOW54O&app_code=TMSIdq5F93WcI7oofW0nhw&waypoint0=47.6610686%2C-122.28275&waypoint1=47.65922%2C-122.31775&lc=1652B4&lw=6&t=0&ppi=320&w=400&h=600


## Weather

In [681]:
import json

def GetWeather(weather):
    CLIENT_ID = ''
    CLIENT_SECRET=''
    
    URL = 'http://api.aerisapi.com/observations/seattle,wa?'+'client_id='+CLIENT_ID+'&client_secret='+CLIENT_SECRET
    r = requests.get(url=URL)    
    soup = json.loads(bs4.BeautifulSoup(r.text, 'lxml').html.body.get_text())['response']
    general = soup['ob']['weather'].lower()
    humid = soup['ob']['humidity']
    temp = soup['ob']['tempF']
    precip = soup['ob']['precipIN']
    wind = soup['ob']['windMPH']
    snwd = soup['ob']['snowDepthIN']
    print(general, humid, temp, precip, wind, snwd)

In [682]:
GetWeather('string')

mostly clear 72 59 0 3 None


## Get Wifi

In [926]:
import pandas as pd
wifi = pd.read_csv('./freeWifi.csv', encoding='ISO-8859-1', header=None)

name = list(wifi[0])
address = list(wifi[1])
wifi = []
for i in range(len(ls)):
    a = GetLatLon(ls[i])
    wifi.append((name[i], address[i], a))

In [937]:
def GetWifi():
    wifi = [('Starbucks', '1912 Pike Pl', ('49.67807', '-124.99319')),
 ('Starbucks Coffee Works', '107 Pike St', ('45.3637143', '-75.6266729')),
 ('Barnes & Noble', '600 Pine St, #107', ('49.15622', '-123.9398822')),
 ('The Roosevelt Hotel', '1531 7th Ave', ('49.2060616', '-122.9373887')),
 ('Grand Hyatt Seattle', '721 Pine St', ('48.43624', '-123.39273')),
 ('Six Arms', '300 E Pike St', ('45.3625104', '-75.6257203')),
 ('Bauhaus Books & Coffee', '301 E Pine St', None),
 ('Starbucks', '1125 4th Ave', ('49.3761636', '-121.4357941')),
 ('Seattle Ferry Terminal ', '801 Alaskan Way', None),
 ('Seattle Public Library', '1000 4th Ave', ('49.2065678', '-122.92326')),
 ('Renaissance Seattle Hotel',
  '515 Madison St',
  ('49.8912551', '-97.2033325')),
 ('Caffe Vita', '1005 E Pike St', ('38.9861', '-84.82811')),
 ('Caffe Umbria', '320 Occidental Ave. S', ('22.1474514', '-101.0415156')),
 ('Café Presse', '1117 12th Ave', ('50.5096884', '-104.61769')),
 ('Skillet Diner', '1400 E Union St', ('43.95679', '2.88693')),
 ('Zoka Coffee', '129 Central Way', None),
 ('Apple Store', '213 Bellevue Sq', ('50.56245', '5.62475')),
 ('Hyatt Regency Bellevue', '900 Bellevue Way NE', ('41.13647', '-95.89249')),
 ('Café Cesura', '1015 108th Ave NE', ('50.3985924', '-105.5089704')),
 ('Pro Sports Club', '4455 148th Ave NE', None),
 ('Panera Bread', '4004 Factoria Blvd SE', None),
 ('Seattle- Tacoma International Airport',
  '17801 International Blvd',
  ('-12.15885', '-76.97894')),
 ('Hilton Seattle Aitport & Conference Center',
  '17620 Ibternational Blvd',
  None),
 ('United Club', 'Concourse A, near Gate 9', None),
 ("Tully's Coffee ", '3080 148th Ave SE, #113', None),
 ('Starbucks', '3181 156th Ave', None),
 ('DoubleTree by Hilton Seattle Airport',
  '18740 International Blvd',
  ('40.75538', '-75.26844')),
 ('Embassy Suits', '3225 158th Ave SE', None),
 ('Hampton Inn', '19445 International Blvd', None),
 ('Nordstorm', '100 Southcenter Shopping', None),
 ('Fairfield Inn Seattle Sea-Tac Airport', '19631 International Blvd', None),
 ('Westfield Southcenter',
  '2800 Southcenter Mall',
  ('39.96928', '-85.3302412')),
 ('Panera Bread', '921 N 10th St, Ste E', ('49.51964', '-115.75179')),
 ("Vince's Coffee Shop",
  '401 Olympia Ave NE, #102',
  ('47.05997', '-122.87741')),
 ('Starbucks', '14022 SE Petrovitsky Rd', None)]
    info = wifi[:3]
    return ('The following resources available are', info)

In [938]:
GetWifi()

('The following resources available are',
 [('Starbucks', '1912 Pike Pl', ('49.67807', '-124.99319')),
  ('Starbucks Coffee Works', '107 Pike St', ('45.3637143', '-75.6266729')),
  ('Barnes & Noble', '600 Pine St, #107', ('49.15622', '-123.9398822'))])

## Get Education

In [921]:
educ = pd.read_csv('./Education.csv', header=None)
educ = educ.values.tolist()

In [919]:
def GetEducation():
    educ = [['King County Library System',
  'Computer Basics Training',
  '960 Newport Way NW, Issaquah, WA, 98027'],
 ['Seattle Public Library',
  'Computer Classes',
  '8635 Fremont Ave N, Seattle, WA, 98103'],
 ['Boys&Girls Clubs of King County',
  'After-school Program',
  '201 19th Ave, Seattle, WA, 98122'],
 ['Coyote Central',
  'Electronics Classes',
  '2300 E Cherry St, Seattle, WA, 98122'],
 ['FAMILYWORKS',
  'Pre-Employment Services',
  '1501 N 45th St, Seattle, WA, 98103'],
 ["GAY CITY: Seattle's LGBTQ Center",
  'Queercore',
  '517 E Pike St, Seattle, WA, 98122'],
 ['Goodwill',
  'Basic Skills& GED Preparation',
  '700 Dearborn Pl S, Seattle, WA, 98144'],
 ['Greenwood Senior Center',
  'Computer Technology Center',
  '525 N 85th St, Seattle, WA, 98103'],
 ['Helping Link',
  'ELS Citizenship Computer Literacy Program',
  '1032 S Jackson St, Suite C, Seattle, WA, 98104'],
 ["Jubulee Women's Center",
  'Computer Basics Training',
  '620 18th Ave E, Seattle, WA, 98112'],
 ['Lambert House', 'Computer Lab', '1818 15th Ave, Seattle, WA, 98122'],
 ['Museum of History & Industry',
  'Maker Day',
  '860 Terry Ave N, Seattle, WA, 98109'],
 ['Phinney Neighborghood Association',
  'Arts, Scheduling and Community Education Programs',
  '6532 Phinney Ave N, Seattle, WA, 98103'],
 ["Seattle Children's Research Institute",
  'STEM High School Internship Program',
  '1900 9th Ave, Seattle, WA, 98101'],
 ['Seattle Indian Center',
  'Employment Services',
  '1265 S Main St, Suite 105, Seattle, WA, 98144'],
 ['Seattle Parks and Recreation - Community Centers',
  'Recreational Technology',
  '2323 E Cherry St, Seattle, WA, 98122'],
 ['United Indians of all Tribes Foundation',
  'Workforce Investment, Employment, Services',
  '5011 Bernie Whitebear Way, Daybreak Star Center, Seattle, WA, 98199'],
 ['Seattle Public Library', 'ESL Classes', '1000 4th Ave, Seattle, WA, 98104'],
 ['Casa Latina', 'ESL Classes', '317 17th Ave S, Seattle, WA, 98144'],
 ['Center for Multicultural Health',
  'Refugee and Immigrant Resources',
  '1120 E Terrace St, Suite 200, Seattle, WA, 98122'],
 ["Children's Home Society of Washington",
  'Classes and Support Group',
  '2611 NE 125th St, Suite 145, Seattle, WA, 98125'],
 ['Chinese Information and Service Center',
  'Immigrant Transition Programs',
  '611 S Lane St, Seattle, WA, 98104'],
 ['Literacy Source',
  'Educational Services',
  '3200 NE 125th St, Seattle, WA, 98125'],
 ['North Seattle College',
  'Pre-College Programs',
  '9600 College Way N, Seattle, WA, 98103'],
 ['Seattle Central College',
  'Division of Basic Studies/ABE/ESL',
  '1701 Broadway, Seattle, WA, 98122'],
 ['Farestart', 'Youth Barista Program', '1818 Yale Ave, Seattle, WA, 98101'],
 ['Trac Associates',
  'Emplyment Services',
  '215 6th Ave N, Suite 100, Seattle, WA, 98109'],
 ['Worksource - King County',
  'Veterans Program',
  '9600 College Way N, The Opportunity Center, Seattle, WA, 98103'],
 ['Church of Jesus Christ',
  'Training',
  '17935 Aurora Ave N, Shoreline, WA, 98133'],
 ['Sunrise Services',
  'Employment - Assessment/Management',
  '17962 Midvale N., #232, Shoreline, WA, 98133'],
 ['Work Opportunities',
  'Rehabilitation',
  '22316 70th Ave W, Mountlake Terrace, WA, 98043'],
 ['Bremerton Housing Authority',
  'Barista Training Program',
  '4650 Bay Vista Blvd, Bremerton, WA, 98312']]
    info = educ[:3]
    return ('The following resources available are', info)

In [920]:
GetEducation()

('The following resources available are',
 [['King County Library System',
   'Computer Basics Training',
   '960 Newport Way NW, Issaquah, WA, 98027'],
  ['Seattle Public Library',
   'Computer Classes',
   '8635 Fremont Ave N, Seattle, WA, 98103'],
  ['Boys&Girls Clubs of King County',
   'After-school Program',
   '201 19th Ave, Seattle, WA, 98122']])